In [1]:
import tensorflow as tf

## Load Dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Model Architecture

In [3]:
# Architecture Parameters
arch_parameters = [
    {
        "layer_type" : "input",
        "layer_namescope" : "model_ip",
        "input_width" : 28,
        "input_height" : 28,
        "input_channels" : 1
    },
    {
        "layer_type" : "conv",
        "layer_namescope" : "model_block_1_conv3_1",
        "layer_num_filters" : 32,
        "layer_kernel_size" : 3,
        "layer_activation" : "relu",
        "layer_strides" : 1,
        "layer_padding" : "valid"
    },
    {
        "layer_type" : "conv",
        "layer_namescope" : "model_block_1_conv3_2",
        "layer_num_filters" : 64,
        "layer_kernel_size" : 3,
        "layer_activation" : "relu",
        "layer_strides" : 1,
        "layer_padding" : "same"
    },
    {
        "layer_type" : "maxpool",
        "layer_pool_size" : (2, 2),
        "layer_namescope" : "model_block_1_max_pool_1",
        "layer_padding" : "valid"
    },
    {
        "layer_type" : "BatchNorm",
        "layer_namescope" : "model_block_1_BatchNorm_1"
    },
    {
        "layer_type" : "conv",
        "layer_namescope" : "model_block_1_conv1_1",
        "layer_num_filters" : 128,
        "layer_kernel_size" : 3,
        "layer_activation" : None,
        "layer_strides" : 1,
        "layer_padding" : "valid"     
    },
    {
        "layer_type" : "relu",
        "layer_namescope" : "model_block_1_relu_1"
    },
    # End of Block 1 
    {
        "layer_type" : "dense",
        "layer_units" : 10,
        "layer_namescope" : "model_dense_final",
        "layer_activation" : "softmax"
    }    
]

In [4]:
class CustomDynamicModel(tf.keras.Model):
    """
        CustomDynamicModel Architecture
    """
    def __init__(self, arch_parameters):
        """
            Initialize the model and all the necessary variables
        """
        super(CustomDynamicModel, self).__init__()
        self.model = None
        
        # Create Architecture
        ip = tf.keras.layers.Input(shape=(arch_parameters[0]["input_width"],
                                                     arch_parameters[0]["input_height"],
                                                     arch_parameters[0]["input_channels"]),
                                        name=arch_parameters[0]["layer_namescope"])
        
        for i, layer in enumerate(arch_parameters[1:-1]):
            if layer["layer_type"] == "conv":
                if i == 0:
                    # Just so that we use ip for the first layer instead of x as a calling argument
                    x = tf.keras.layers.Conv2D(filters=layer["layer_num_filters"], kernel_size=layer["layer_kernel_size"],
                                               activation=layer["layer_activation"], strides=layer["layer_strides"],
                                               padding=layer["layer_padding"], name=layer["layer_namescope"])(ip)
                else:
                    x = tf.keras.layers.Conv2D(filters=layer["layer_num_filters"], kernel_size=layer["layer_kernel_size"],
                                               activation=layer["layer_activation"], strides=layer["layer_strides"],
                                               padding=layer["layer_padding"], name=layer["layer_namescope"])(x)
            elif layer["layer_type"] == "maxpool":
                x = tf.keras.layers.MaxPool2D(padding=layer["layer_padding"], pool_size=layer["layer_pool_size"],
                                              name=layer["layer_namescope"])(x)
            elif layer["layer_type"] == "relu" :
                x = tf.keras.layers.ReLU(name=layer["layer_namescope"])(x)
            elif layer["layer_type"] == "BatchNorm":
                x = tf.keras.layers.BatchNormalization(name=layer["layer_namescope"])(x)
                    
        if arch_parameters[-1]["layer_type"] == "conv":
            out = tf.keras.layers.Conv2D(filters=arch_parameters[-1]["layer_num_filters"],
                                         kernel_size=arch_parameters[-1]["layer_kernel_size"],
                                         activation=arch_parameters[-1]["layer_activation"], 
                                         strides=arch_parameters[-1]["layer_strides"],
                                         padding=arch_parameters[-1]["layer_padding"],
                                         name=arch_parameters[-1]["layer_namescope"])(x)
        elif arch_parameters[-1]["layer_type"] == "dense":
            x = tf.keras.layers.Flatten(name="model_flatten")(x)
            out = tf.keras.layers.Dense(units=arch_parameters[-1]["layer_units"],
                                        activation=arch_parameters[-1]["layer_activation"],
                                        name=arch_parameters[-1]["layer_namescope"])(x)
        
        
        self.model = tf.keras.Model(inputs=ip, outputs=out, name="CustomDynamicModel")
        
    
    def call(self, inputs, training=False):
        """
            Call the model
        """
        return self.model(inputs)

    def summary(self):
        return self.model.summary()

In [5]:
model = CustomDynamicModel(arch_parameters)

In [6]:
model.summary()

Model: "CustomDynamicModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_ip (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
model_block_1_conv3_1 (Conv2 (None, 26, 26, 32)        320       
_________________________________________________________________
model_block_1_conv3_2 (Conv2 (None, 26, 26, 64)        18496     
_________________________________________________________________
model_block_1_max_pool_1 (Ma (None, 13, 13, 64)        0         
_________________________________________________________________
model_block_1_BatchNorm_1 (B (None, 13, 13, 64)        256       
_________________________________________________________________
model_block_1_conv1_1 (Conv2 (None, 11, 11, 128)       73856     
_________________________________________________________________
model_block_1_relu_1 (ReLU)  (None, 11, 11, 128)

## Training

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [8]:
model.fit(
    x=x_train, y=y_train,
    batch_size=32,
    epochs=10, validation_data=(x_test,y_test)
)

Epoch 1/10
1875/1875 [==============================] - 40s 4ms/step - loss: 0.1800 - sparse_categorical_accuracy: 0.9443 - val_loss: 0.0364 - val_sparse_categorical_accuracy: 0.9884
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0357 - sparse_categorical_accuracy: 0.9895 - val_loss: 0.0451 - val_sparse_categorical_accuracy: 0.9871
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0244 - sparse_categorical_accuracy: 0.9924 - val_loss: 0.0430 - val_sparse_categorical_accuracy: 0.9883
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0157 - sparse_categorical_accuracy: 0.9949 - val_loss: 0.0442 - val_sparse_categorical_accuracy: 0.9882
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0144 - sparse_categorical_accuracy: 0.9955 - val_loss: 0.0372 - val_sparse_categorical_accuracy: 0.9903
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0126 - spar